In [1]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()

True

In [15]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [4]:
! pip install wikipedia faiss-cpu

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached faiss_cpu-1.11.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.0 MB 1.9 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/15.0 MB 1.5 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/15.0 MB 1.5 MB/s eta 0:00:10
   -- -------------------------------

In [27]:
from langchain_community.retrievers import WikipediaRetriever

retriever=WikipediaRetriever(top_k_results=2,lang="en")

In [30]:
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(query="Cristiano Ronaldo", lang="en")
docs = loader.load()

print(docs[0].page_content)

d:\Langchain_models\venv_genai\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\Langchain_models\venv_genai\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu] ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al-Nassr and the Portugal national team. Nicknamed CR7, he is widely regarded as one of the greatest players in history, and has won numerous individual accolades throughout his professional footballing career, including five Ballon d'Or awards, a record three UEFA Men's Player of the Year Awards, four European Golden Shoes, and was named five times the world's best player by FIFA. He has won 34 trophies in his career, including seven league titles, five UEFA Champions Leagues and the UEFA European Championship. Ronaldo holds the records for most goals (140) and assists (42) in the Champions League, goals (14) and assists (8) in the European Championship, and most international appearances (221) and international goals (138). He is one of the few players to have made over 1

In [31]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever(top_k_results=2, lang="en")
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [32]:
docs

[Document(metadata={'title': 'Dominion of Pakistan', 'summary': 'The Dominion of Pakistan, officially Pakistan, was an independent federal dominion in the British Commonwealth of Nations, which existed from 14 August 1947 to 23 March 1956. It was created by the passing of the Indian Independence Act 1947 by the British parliament, which also created an independent Dominion of India.\nThe new dominion consisted of those presidencies and provinces of British India which were allocated to it in the Partition of India. Until 1947, these regions had been ruled by the United Kingdom as a part of the British Empire.\nIts status as a federal dominion within the British Empire ended in 1956 with the completion of the Constitution of Pakistan, which established the country as a republic. The constitution also administratively split the nation into West Pakistan and East Pakistan. Until then, these provinces had been governed as a singular entity, despite being separate geographic exclaves. Event

In [33]:
for i, doc in enumerate(docs):
    print(f"\n--- Result{i+1} ---")
    print(f"Content:\n{doc.page_content}...")


--- Result1 ---
Content:
The Dominion of Pakistan, officially Pakistan, was an independent federal dominion in the British Commonwealth of Nations, which existed from 14 August 1947 to 23 March 1956. It was created by the passing of the Indian Independence Act 1947 by the British parliament, which also created an independent Dominion of India.
The new dominion consisted of those presidencies and provinces of British India which were allocated to it in the Partition of India. Until 1947, these regions had been ruled by the United Kingdom as a part of the British Empire.
Its status as a federal dominion within the British Empire ended in 1956 with the completion of the Constitution of Pakistan, which established the country as a republic. The constitution also administratively split the nation into West Pakistan and East Pakistan. Until then, these provinces had been governed as a singular entity, despite being separate geographic exclaves. Eventually, the East became Bangladesh and the

## vector store retriever

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [35]:
documents=[
    Document(page_content='langchain help developers to build LLM applications easily.'),
    Document(page_content='Chroma is a vector database optimised for LLM based search'),
    Document(page_content='Embeddings convert text into high-dimensional vectors'),
    Document(page_content='LLM foundation models can create embeddings once the text is provided. also, OpenAI is one of the bestperfoming in all of them'),
]

In [36]:


embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

d:\Langchain_models\venv_genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
vectorstore=Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name='my_collection'
)

In [38]:
retriever=vectorstore.as_retriever(search_kwargs={'k':2})

In [39]:
query="why is chroma used?"
results=retriever.invoke(query)

In [40]:
for i,doc in enumerate(results):
    print(f"\n--- results {i+1} ---")
    print(doc.page_content)


--- results 1 ---
Chroma is a vector database optimised for LLM based search

--- results 2 ---
langchain help developers to build LLM applications easily.


## Maximal Marginal Relevance (MMR)

In [4]:
docs=[
    Document(page_content="Langchain makes it easy to work with LLMs"),
    Document(page_content="langchain is used to build LLM applications"),
    Document(page_content='chroma is used to store and search document embeddings'),
    Document(page_content="MMR helps us to get diverse results when doing similarity search"),
    Document(page_content="Langchain supports Chroma, FAISS, Pinecone and more other vector stores and databases"),

]

In [5]:
from  langchain_community.vectorstores import FAISS 

embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

vectorstore=FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

d:\Langchain_models\venv_genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
retriever= vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3,"lambda_mult":1}
)

In [7]:
query="what is langchain?"
results=retriever.invoke(query)

In [8]:
for i, doc in enumerate(results):
    print(f"\n--- results {i+1} ---")
    print(doc.page_content)
    



--- results 1 ---
langchain is used to build LLM applications

--- results 2 ---
Langchain supports Chroma, FAISS, Pinecone and more other vector stores and databases

--- results 3 ---
Langchain makes it easy to work with LLMs
